In [49]:
import numpy as np

# Neural Network (Softmax) from Scratch

In this document, I implement a neural network from scratch using python, numpy, and mathematics. While deep learning frameworks allow users to ignore many of the technical details of neural networks, an understanding of the mathematics behind deep learning could allow for an increased level of intuition on the subject. 

# Activation Functions

In this section, I define a few activation functions and their derivatives. I will use ReLU activation for hidden layers and Softmax activation for the output layer, but I also wish to define a few other commonly-used functions.

I begin with the commonly-used ReLU activation.

In [12]:
def relu(X):
    return np.maximum(0,X)

There also exists a "Leaky" ReLU, which is used to prevent the activation from "dying."

Consider inputs with large negative bias: the standard ReLU will consistently return 0. Thus, this ReLU neuron will not be contributing to the network through updating weights. The Leaky ReLU attempts to fix this problem. 

Note: the 0.01 term in the l_relu function could itself be a parameter.

In [13]:
def l_relu(X):
    if X < 0:
        return X*0.01
    else:
        return X

The Sigmoid activation is commonly used in the output layer of a two-class logistic regression network.

In [14]:
def sigmoid(X):
    return 1.0/(1+np.exp(-X))

The Tanh activation is a rescaled sigmoid, allowing for stronger gradients. 

In [15]:
def tanh(X):
    return np.tanh(X)

The Softmax activation is commonly used in the output layer of multi-class classification networks. For the purpose of this high-level document, I directly implement its cross-entropy loss function and gradient. 

In [16]:
def softmax(X):
    return np.exp(X)/np.sum(np.exp(X), axis = 0)

def softmax_error(Y,A):
    return np.sum(Y * np.log(A))

def softmax_grad(Y,A):
    return A - Y

# Initialization of Parameters

Each layer of the network contains a number of neurons. I use a list num_neurons to represent these numbers. Thus, num_neurons[0] would represent the number of input features, num_neurons[1] would be the number of neurons in the first hidden layer, etc.

The function initialize() returns a dictionary containing initialized values of W and b for each layer. W and b are the weight and bias parameters respectively. In particular, we want small (positive or negative) values for the weights, and 0s for the biases. 

In [17]:
def initialize(num_neurons):
    
    parameters = {}
    
    for l in range(1, len(num_neurons)):
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros([layers_dims[l], 1])
        
    return parameters

# Forward Propagation

In the forward propagation step, we use the network input (A), initialized parameters (parameters), and the number of layers (L). We save a cache "Z_A_back" of Z values (WA + b) and their activations (Z) to be used in the back propagation step. We also return the activation of the final layer (A). 

In [18]:
def  forward_prop(A, parameters, L):
    
    Z_A_back  = {}
    
    for l in range(1, L):
        W = parameters['W' + str(l)]
        b = parameters['b' + str(l)]
        Z = np.dot(W, A) + b
        if l < (L-1):
            A = relu(Z)
        else:
            A = softmax(Z)
        Z_A_back['Z' + str(l)] = Z
        Z_A_back['A' + str(l)] = A
        
    return A, Z_A_back

# Computing Costs

In this step, we use the activation of the final layer (AL) and the label vector for our examples (Y). We use Y to find the number of examples (m), and then use m, along with the softmax error, to compute the cross-entropy cost. 

In [40]:
def costs(AL, Y):
    
    m = Y.shape[1]
    cost = (-1/m) * softmax_error(Y, AL)
    cost = np.squeeze(cost)
    
    return cost

# Back Propagation

Inputs are network input (X), the label vector (Y), the activation of the final layer (AL), the back prop cache (Z_A_back), the weight/bias parameters (parameters), and the number of layers (L). 

The function back_prop() calculates the gradient for each parameter and stores them in a dictionary (grads), which is returned. This involves using the softmax gradient for the first step (last layer). 

In [39]:
def back_prop(X, Y, AL, Z_A_back, parameters, L):
    
    m = Y.shape[1]
    grads = {}
    grad_Z =  (1/m) * softmax_grad(Y, AL)
    
    for l in reversed(range(1, L-1)):
        grads['W' + str(l+1)] = np.dot(grad_Z, Z_A_back['A' + str(l)].T)
        grads['b' + str(l+1)] = np.sum(grad_Z, axis=1, keepdims=True)        
        grad_A = np.dot(parameters['W' + str(l+1)].T, grad_Z)
        grad_A[Z_A_back['Z' + str(l)] <= 0] = 0 #Note: this step, along with the next, correspond to the back prop of ReLU 
        grad_Z = grad_A

    grads['W1'] = np.dot(grad_Z, X.T)
    grads['b1'] = np.sum(grad_Z, axis=1, keepdims=True)
    
    return grads

# Updating the Parameters 

Inputs are the parameters dictionary (parameters), the gradient dictionary (grads), the learning rate (rate), and the number of non-input laters (L).

The function update_parameters() uses gradient descent to update the weight and bias parameters. It returns the updated parameters dictionary. 

In [21]:
def update_parameters(parameters, grads, rate, L):
    for l in reversed(range(1, L-1)):
        parameters['W' + str(l)] -= rate * grads['W' + str(l)]
        parameters['b' + str(l)] -= rate * grads['b' + str(l)]
        
    return parameters

# Creating the Model

The neural network model can now be created by using the above functions.

In [22]:
def Model(X, Y, X_test, Y_test, num_neurons, iterations, rate, L):
    
    parameters = initialize(num_neurons)
    training_costs = []
    testing_costs = []
    
    for iteration in range(iterations+1):
        AL, caches = forward_prop(X, parameters, L)
        cost = costs(AL, Y)
        grads = back_prop(X, Y, AL, Z_A_back, parameters, L)
        parameters = update_parameters(parameters, grads, rate, L)
        AL_test, _ = forward_prop(X_test, parameters, L)
        cost_test = costs(AL_test, Y_test)
        training_costs.append(cost)
        testing_costs.append(cost_test)
        if iteration % 100 == 0:
            print('Iteration number:', iteration, 'Training Cost:', cost, '& Testing Cost:', cost_test)
    
    return parameters

# Predicting

Predictions can be made using the computed parameters. 

Softmax returns an n-by-1 array, where n is the number of categories. The top element corresponds to model-calculated probability of category 0, the second element is the probability for category 1, etc.

The function predict() takes the argmax of this array to predict the category with the greatest probability. 

In [23]:
def predict(X, parameters):

    AL, _ = forward_prop(X, parameters)
    predictions = np.argmax(AL, axis = 0)
    
    return predictions

# Final Notes

In order for a dataset to be usable with this model, the data must follow a specific format. In particular, X must be an n_x by m matrix, where n_x is the number of features and m is the number of data points. Further, Y must be an n_y by m matrix, where n_y is the number of classification labels and m is the number of data points. A function such as np.eye() may be useful when converting common label formats to the correct format for this model.

